# Задание 1


### Целью этого задания является знакомство со стандартными контейнерами и некторыми функциями из стандартных библиотек для машинного обучения.

Напишите наивный байесовский классификатор и сравните его с реализацией NaiveBayesClassifier из библиотеки nltk.

Написанный вами классификатор должен обладать следубщими свойствами:
<ul>
<li>В предложенном интерфейсе класса должны быть реализованы все методы и все поля. Для их хранения предподсчитанных данных рекомендуется использовать контейнеры Counter или defaultdict из библиотеки collections. Для предсказания категории рекомендуется использовать numpy.</li>
<li>Должна использоваться модель, предложенная в теории.</li>
<li>Точность предсказаний не менее <b>0.9</b>!</li>
<li>После реализации класса протестируйте его с помощью кроссвалидации с k=10. Рекомендуется использовать класс KFold из библиотеки sklearn.</li>
<li>Постройте постройте диаграмму размаха для классификаторов (своего и из библиотеки).</li>
</ul>

### Теория

Теория находится в файле problems1-theory.pdf

# Решение

In [1]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from nltk import NaiveBayesClassifier
from matplotlib import pyplot as plt

import math
import time

%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Прочитайте данные из файла

In [2]:
data_path = "ham-spam.csv"

In [3]:
def get_data_from_file(data_path):

    file_data = open(data_path)

    data = []
    for string in file_data:
        comma = string.find(',')
        subject = comma + string[comma:].find(':') + 2
        sn = -1
        data.append([string[:comma], string[subject:sn]])

    #print(data[:4])

    file_data.close()
    
    return data

def get_X_and_Y_Train(data_path):
    
    data = get_data_from_file(data_path)[1:]
    X_Train = [x[1] for x in data]
    Y_Train = [y[0] for y in data]
    
    return X_Train, Y_Train

In [4]:
X_Train, Y_Train = get_X_and_Y_Train(data_path)

print(X_Train[0])

re : 2 . 882 s - > np np > date : sun , 15 dec 91 02 : 25 : 02 est > from : michael < mmorse @ vm1 . yorku . ca > > subject : re : 2 . 864 queries > > wlodek zadrozny asks if there is "" anything interesting "" to be said > about the construction "" s > np np "" . . . second , > and very much related : might we consider the construction to be a form > of what has been discussed on this list of late as reduplication ? the > logical sense of "" john mcnamara the name "" is tautologous and thus , at > that level , indistinguishable from "" well , well now , what have we here ? "" . to say that ' john mcnamara the name ' is tautologous is to give support to those who say that a logic-based semantics is irrelevant to natural language . in what sense is it tautologous ? it supplies the value of an attribute followed by the attribute of which it is the value . if in fact the value of the name-attribute for the relevant entity were ' chaim shmendrik ' , ' john mcnamara the name ' would be fals

### Реализуйте все методы в классе NaiveBayes

In [5]:
GLOBAL_PRINT = True

def mprint(*arg):
    if GLOBAL_PRINT:
        print(*arg)

In [6]:
class NaiveBayes(object):
    """
    Наивный байесовский классификатор.
    Для каждого входного сообщения слово учитывается один раз при расчете итоговой вероятности.

    Parameters
    ----------
    category_priors : default | None, optional, default None
        Априорные вероятности категорий.
        Если None, то классификатор должен сам их вычислить.

    weight : float, optional, default 1
        Вес одного слова в формуле взвешенной вероятности

    supposed_prob : float, optional, default 0.5
        Предполагаемая вероятность слова в категории
    """
    
    def __init__(self, category_priors=None, weight=1, supposed_prob=0.5):
        self.category_priors = category_priors
        self.weight = weight
        self.supposed_prob = supposed_prob

        # Количество отдельных слов в заданной категории
        self.feature_category_counts = defaultdict(lambda: defaultdict(int))

        # Количество всех документов в данной категории
        self.category_doc_counts = Counter()

        # Количество встреч слова во всех сообщениях
        self.feature_counts = Counter()
        
        # Все категории сообщений
        self.categories = defaultdict()
        
        # Количество всех слов во всех сообщениях (считая, что в каждом сообщении все слова - уникальные)
        self.total = 0
        
        # Обучен ли классификатор
        self.isFit = False

    def fit(self, x_train, y_train):
        """
        Производит обучение наивного байесовского классификатора.

        Parameters
        ----------
        text : list of list of str | list of str | str
            Входной текст описывается строкой, которую будет токенизирована по пробелу.
            Если строка не токенизирована, то текст должен быть токенизирован.
            Может быть передано несколько сообщений, которые будут токенезированы, если необходимо.

        y_train : list of str
            содержит список меток (названий категорий) для сообщений из x_train

        Returns
        -------
        self : object
            Returns self
        """
        # Подсчитываем количество категорий, документов и слов в каждой категории
        # и количество встреч слова во всех сообщениях
        
        ts = time.time()
        
        if x_train.__class__ == str:
            x_train = [x_train]
        
        for y in y_train:
            self.category_doc_counts[y] += 1
            
        self.categories = self.category_doc_counts.keys()
        
        for x_ind in range(len(x_train)):
            if x_train[x_ind].__class__ == str:
                msg = x_train[x_ind].split(' ')
            else:
                msg = x_train[x_ind]
            
            for word_ind in range(len(msg)):
                if msg.count(msg[word_ind]) > 1 and msg.index(msg[word_ind]) != word_ind:
                    continue
                    
                self.feature_counts[msg[word_ind].lower()] += 1
                
                self.feature_category_counts[msg[word_ind].lower()][y_train[x_ind]] += 1
        
        self.total = 0
        for w in self.feature_counts:
            self.total += self.feature_counts[w]
        
        # Если априорные вероятности категорий не заданы, то надо аппроксимировать их
        if self.category_priors is None:
            self.category_priors = defaultdict()
            s = 0 #можно ли это написать с помощью sum(self.category_doc_counts.values())?
            for x in self.category_doc_counts.values():
                s += x
            for cat in self.categories:
                self.category_priors[cat] = self.category_doc_counts[cat] / s

        tf = time.time()
        mprint('Fit time:', (tf-ts) // 60, 'minutes and ', (tf-ts) % 60, 'seconds')
        
        self.isFit = True
        return self
    
    def predict(self, text):
        """
        Предсказывает метки категорий для text.

        Parameters
        ----------
        text : list of list of str | list of str | str
            Входной текст описывается строкой, которую будет токенизирована по пробелу.
            Если строка не токенизирована, то текст должен быть токенизирован.
            Может быть передано несколько сообщений, которые будут токенезированы, если необходимо.

        Returns
        -------
        categories : list of str
            Возвращает названия категорий для text.
        """
        
        if not self.isFit:
            return
        
        if text.__class__ == str:
            text = [text]
            
        categories = []
        for t_ind in range(len(text)):
            if text[t_ind].__class__ == str:
                msg = text[t_ind].split(' ')
            else:
                msg = text[t_ind]
            
            p_categories = self.get_probs(msg)
                
            cat_answer = None
            for cat in p_categories:
                if cat_answer is None:
                    cat_answer = cat
                    break
                if p_categories[cat] > p_categories[cat_answer]:
                    cat_answer = cat    

            categories.append(cat_answer)
            
        return categories

    def score(self, text, labels):
        """
        Возвращает точность предсказаний на text для правильных категорий labels.

        Parameters
        ----------
        text : list of list of str | list of str | str
            Входной текст описывается строкой, которую будет токенизирована по пробелу.
            Если строка не токенизирована, то текст должен быть токенизирован.
            Может быть передано несколько сообщений, которые будут токенезированы, если необходимо.
        labels : list of str
            Список категорий для каждого токена из text.

        Returns
        -------
        acc : float
            Точность предсказания.
        """
        if not self.isFit:
            return
        
        return accuracy_score(self.predict(text), labels)

    def get_probs(self, text):
        """
        Считает вероятности принадлежности текста (text) к каждой из категорий

        Parameters
        ----------
        text : list of str | str
            Входной текст описывается строкой, которую будет токенизирована по пробелу.
            Если строка не токенизирована, то текст должен быть токенизирован.

        Returns
        -------
        probs : list of float
            Возвращает вероятности probs всех категорий для текста text
            в порядке их следования в self.category_doc_counts.
        """
        
        if not self.isFit:
            return
        
        # Токенизируем текст, если это необходимо
        
        if text.__class__ == str:
            msg = text.split(' ')
        else:
            msg = text
        
        def mpow(n, base=len(self.categories)):
            if n > 20:
                return inf
            return base**n
        
        p_categories = defaultdict(lambda: float())
        for cat in self.categories:
            p_categories[cat] = mpow(self.get_category_prob(cat, msg))
        
        return p_categories

    def get_category_prob(self, cat, text):
        """
        Считает логарифм вероятность принадлежности сообщения text к категории cat.

        Parameters
        ----------
        cat : str
            Название категории.

        text : list of str
            Список из слов.

        Returns
        -------
        log_prob : float
            Возвращает логарифм вероятности категории cat для текста text.
        """
        
        if not self.isFit:
            return
                
        def mlog(n, base=len(self.categories)):
            if n == 0:
                return 0
            return math.log(n, base)
        
        p_cat = mlog(self.category_priors[cat])
        for word in text:
            p_cat += mlog(self.get_weighted_feature_prob(cat, word))
        
        return p_cat

    def get_weighted_feature_prob(self, cat, feature):
        """
        Вычисляет взвешенную вероятность P(Слово|Категория).

        Parameters
        ----------
        cat : str
            Название категории.

        feature : str
            Слово из текста.

        Returns
        -------
        prob : float
            Возвращает взвешенную вероятность слова feature при условии категории cat.
        """
        
        if not self.isFit:
            return
        
        p_w_i_s = self.feature_category_counts[feature][cat] / self.category_doc_counts[cat]
        p_cat = 1 / len(self.categories)
        a = self.weight * p_cat + self.total * p_w_i_s
        b = self.weight + self.total
        return a / b

    def get_categories(self):
        """
        Возвращает список названий всех категорий.
        Returns
        -------
        cat_list : list of str
        """
        
        if not self.isFit:
            return
        
        return self.categories

In [7]:
nb = NaiveBayes()

#nb.fit(X_Train, Y_Train)

mprint('ham:', nb.category_doc_counts['ham'])
mprint('spam:', nb.category_doc_counts['spam'])

mprint(nb.category_doc_counts.keys())
mprint('get', nb.feature_counts['get'])

mprint('percent', nb.get_category_prob('ham', X_Train[243]))
mprint('percent', nb.get_category_prob('spam', X_Train[243]))

mprint(nb.feature_category_counts['get'])

mprint(nb.category_priors)

ham: 0
spam: 0
dict_keys([])
get 0
percent None
percent None
defaultdict(<class 'int'>, {})
None


In [8]:
def test_myNB():
    
    kf = KFold(n_splits=10, shuffle=True)

    X = np.array(X_Train)
    Y = np.array(Y_Train)

    for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        nb = NaiveBayes()
        nb.fit(X_train, y_train)
        print(nb.score(X_test, y_test))
        
test_myNB()

Fit time: 1.0 minutes and  50.2456419467926 seconds
0.837931034483
Fit time: 1.0 minutes and  41.33989977836609 seconds
0.848275862069
Fit time: 1.0 minutes and  43.44826698303223 seconds
0.831034482759
Fit time: 1.0 minutes and  49.10855579376221 seconds
0.83044982699
Fit time: 1.0 minutes and  46.38993811607361 seconds
0.826989619377
Fit time: 3.0 minutes and  34.54800796508789 seconds
0.851211072664
Fit time: 1.0 minutes and  26.241960048675537 seconds
0.83044982699
Fit time: 1.0 minutes and  30.21858310699463 seconds
0.795847750865
Fit time: 1.0 minutes and  24.656397104263306 seconds
0.861591695502
Fit time: 2.0 minutes and  8.112924098968506 seconds
0.823529411765


Значения классификатора не достигают заявленных в условии 90%, однако после сравнения с результатом nltk я принял решение не улучшать его. (См. комментарий под запуском nltk.)

P.S. На некоторых запусках результат был стабильно 85-87%.

### Сравните вашу реализацию и реализацию из библиотеки nltk

Для использования классификатора из библиотеки не забудьте предподготовить данные. Для подсчета точности этого классификатора можете использовать accuracy_score из метрик sklearn. Для подсчета точности предсказаний вашего классификатора используйте функцию score, которую вы опишете.

In [9]:
# Предобработка данных для классификатора nltk, если требуется
from nltk.tokenize import word_tokenize
from itertools import chain

def get_data_for_nltk(X, y):
    
    training_data = [*zip(X, y)]
    
    vocabulary = set(chain(*[word_tokenize(i[0].lower()) for i in training_data]))

    feature_set = [({i:(i in word_tokenize(sentence.lower())) for i in vocabulary}, tag) for sentence, tag in training_data[:2]]
    
    #print(feature_set)
    return feature_set, vocabulary

def get_data_for_test(test_sentence, vocabulary):
    test_sentence = "This is the best band I've ever heard!"
    featurized_test_sentence =  {i:(i in word_tokenize(test_sentence.lower())) for i in vocabulary}

    return featurized_test_sentence

In [10]:
%%time

# Используйте процедуру KFold для проверки качества классификаторов
kfc = KFold(n_splits=10, shuffle=True)

X = np.array(X_Train[:1500]) #ибо уж очень долго работало
Y = np.array(Y_Train[:1500])

for train_index, test_index in kfc.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    
    ts = time.time()
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    print('Start train')
    
    data, vocabulary = get_data_for_nltk(X_train, y_train)
    nbc = NaiveBayesClassifier.train(data)
    
    print('Finish train')
    
    tf = time.time()
    mprint('NBC Fit time:', (tf-ts) // 60, 'minutes and ', (tf-ts) % 60, 'seconds')
    
    ts = time.time()
    
    cats = []
    for test in X_test:
        cats.append(nbc.classify(get_data_for_test(test, vocabulary)))
        
    print('Score:', accuracy_score(cats, y_test))
    
    tf = time.time()
    mprint('NBC Classify time:', (tf-ts) // 60, 'minutes and ', (tf-ts) % 60, 'seconds')
    
    break
    
    #print(nbc.score(X_test, y_test))

Start train
Finish train
NBC Fit time: 1.0 minutes and  50.3898708820343 seconds
Score: 0.833333333333
NBC Classify time: 18.0 minutes and  27.780794143676758 seconds
CPU times: user 19min 42s, sys: 9.96 s, total: 19min 52s
Wall time: 20min 18s


Здесь всего одна итерация, так как на неё было затрачено порядка 22 минут (то есть на весь цикл уйдёт порядка 4 часов), а результат классификатора nltk несильно отличается от того, что написан мной.

Пример на случай запуска кода:

![png](https://gyazo.com/608c2e8fa881b6b9259289113f365c8f.png)

### Постройте графики размаха для двух классификаторов на одной фигуре.

Рекомендуется использовать встроенные функции построения графиков в pandas.

In [11]:
# пока я не понял, как получить эти графики из исходных данных